In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T
import generate_df as G
from IPython.display import display, HTML
from tabulate import tabulate
from unidecode import unidecode

In [2]:
spark = SparkSession.builder.appName("finance_data_processing")\
.config("spark.jars","/home/daiane/Downloads/postgresql-42.7.3.jar").getOrCreate()

df_demonstracao_result = G.table_to_df("finance_raw_data", "demonstracao_resultado_receita_lucro", spark)
df_demonstracao_result.persist()

# df_temp = df_demonstracao_result_.withColumn("data", F.to_date(F.col("data"), "MM/yyyy"))

# last_date = df_temp.agg(F.max("data")).collect()[0][0]

# last_date_str = last_date.strftime("%m/%Y")


# df_demonstracao_result = df_demonstracao_result_.filter(F.col("data") == last_date_str)
# df_demonstracao_result.persist()

# print(last_date)
# print(last_date_str)


# dfSample_demonstracao_result = df_demonstracao_result.sample(False, 0.1).limit(30)
# table = tabulate(dfSample_demonstracao_result.collect(), headers=dfSample_demonstracao_result.columns, tablefmt='html')

# display(HTML(table))
# print(dfSample_demonstracao_result.count())
# dfSample_demonstracao_result.printSchema()



24/07/18 17:05:44 WARN Utils: Your hostname, victor-Lenovo-ideapad-330-15IKB resolves to a loopback address: 127.0.1.1; using 192.168.1.74 instead (on interface wlp2s0)
24/07/18 17:05:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/07/18 17:05:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/18 17:05:59 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


DataFrame[instituicao: string, codigo: string, conglomerado: string, codigo_conglomerado_financeiro: string, codigo_conglomerado_prudencial: string, tcb: string, tc: string, ti: string, cidade: string, uf: string, data: string, rendas_operacoes_credito_a1: string, rendas_operacoes_arrendamento_mercantil_a2: string, rendas_operacoes_com_tvm_a3: string, rendas_operacoes_instrumentos_financeiros_derivativos_a4: string, resultado_operacoes_cambio_a5: string, rendas_aplicacoes_compulsorias_a6: string, receitas_intermediacao_financeira_soma_a1_a2_a3_a4_a5_a6: string, despesas_captacao_b1: string, despesas_obrigacoes_por_emprestimos_e_repasses_b2: string, despesas_operacoes_arrendamento_mercantil_b3: string, resultado_operacoes_cambio_b4: string, resultado_de_provisao_para_creditos_dificil_liquidacao_b5: string, despesas__intermediacao_financeira_soma_b1_b2_b3_b4_b5: string, resultao_intermediacao_financeira_c_soma_a_b: string, rendas_prestacao_servico_d1: string, rendas_tarifas_bancarias_d2:

In [3]:
df_info_cadastral = G.table_to_df("finance_raw_data", "info_cadastral_entidades", spark)
df_info_cadastral.persist()

# dfSample_info_cadastral_entidades = df_info_cadastra l.sample(False, 0.1).limit(30)
# table = tabulate(dfSample_info_cadastral_entidades.collect(), headers=dfSample_info_cadastral_entidades.columns, tablefmt='html')
# display(HTML(table))

# df_info_cadastral.count()
# dfSample_info_cadastral_entidades.printSchema()


DataFrame[data: string, codigo_cadastro_bacen: string, codigo_sisbacen: string, codigo_pais_sede: string, nome_pais_sede: string, nome_uf_sede: string, codigo_municipio_sede: string, nome_municipio_sede: string, nome_entidade: string, nome_entidade_nao_formatado: string, cnpj: string, cnpj_raiz: string, codigo_situacao: string, descricao_situacao: string, codigo_tipo_entidade_segmento: string, nome_tipo_entidade: string, codigo_natureza_juridica: string, descricao_natureza_juridica: string, codigo_esfera_publica: string, nome_reduzido: string, sigla_entidade: string, nome_fantasia: string, empresa_publica: string]

In [4]:
df_info_cadastral_select = df_info_cadastral.select("cnpj", "data", "cnpj_raiz", "codigo_sisbacen", "nome_tipo_entidade", "descricao_situacao", "nome_entidade", "nome_fantasia")

# a = df_info_cadastral_select.sample(False, 0.1).limit(50)
# table = tabulate(a.collect(), headers=a.columns, tablefmt='html')
# df_info_cadastral_select.filter(df_info_cadastral_select["cnpj"].isNull()).count()
# display(HTML(table))


### Duplicação

In [10]:
# df_info_cadastral

columns_info = df_info_cadastral_select.columns
columns_info.remove("data")

df_unique_info_ = df_info_cadastral_select.dropDuplicates(subset=columns_info)

# df_demonstracao_result

columns_demonstracao = df_demonstracao_result.columns
columns_demonstracao.remove("data")

df_formatado = df_demonstracao_result.withColumnRenamed("data", "data_trimestral").withColumn("codigo", F.lpad(df_demonstracao_result["codigo"], 8, '0'))

df_unique_demonstracao_ = df_formatado.dropDuplicates(subset=columns_demonstracao)

df_unique_info_.printSchema()
df_unique_demonstracao_.printSchema()



root
 |-- cnpj: string (nullable = true)
 |-- data: string (nullable = true)
 |-- cnpj_raiz: string (nullable = true)
 |-- codigo_sisbacen: string (nullable = true)
 |-- nome_tipo_entidade: string (nullable = true)
 |-- descricao_situacao: string (nullable = true)
 |-- nome_entidade: string (nullable = true)
 |-- nome_fantasia: string (nullable = true)

root
 |-- instituicao: string (nullable = true)
 |-- codigo: string (nullable = true)
 |-- conglomerado: string (nullable = true)
 |-- codigo_conglomerado_financeiro: string (nullable = true)
 |-- codigo_conglomerado_prudencial: string (nullable = true)
 |-- tcb: string (nullable = true)
 |-- tc: string (nullable = true)
 |-- ti: string (nullable = true)
 |-- cidade: string (nullable = true)
 |-- uf: string (nullable = true)
 |-- data_trimestral: string (nullable = true)
 |-- rendas_operacoes_credito_a1: string (nullable = true)
 |-- rendas_operacoes_arrendamento_mercantil_a2: string (nullable = true)
 |-- rendas_operacoes_com_tvm_a3: s

### Remover acentos

In [6]:
def clean_accent_(texto):
    return unidecode(texto) if texto else None

clean_accent = F.udf(clean_accent_, F.StringType())

df_unique_info = df_unique_info_.select([F.upper(F.trim(clean_accent(F.col(column)))).alias(column) for column in df_unique_info_.columns])

df_unique_demonstracao = df_unique_demonstracao_.select([F.upper(F.trim(clean_accent(F.col(column)))).alias(column) for column in df_unique_demonstracao_.columns])

In [9]:
df_join_ = df_unique_info.join(df_unique_demonstracao, df_unique_info["cnpj_raiz"] == F.trim(df_unique_demonstracao["codigo"]), "full_outer")\
.select(df_unique_demonstracao["instituicao"], df_unique_demonstracao["tc"], df_unique_demonstracao["conglomerado"], df_unique_demonstracao["codigo"],\
        *df_unique_info.columns)

df_join = df_join_.dropDuplicates()

# Filtrar registros que estão apenas no DataFrame "demonstracao_result"
only_demonstracao_result = df_join.filter(
    F.col("nome_entidade").isNull() &
    F.col("instituicao").isNotNull() &
    (F.trim(F.col("instituicao")) != "")
)

# Filtrar registros que estão apenas no DataFrame "info_cadastral"
only_info_cadastral = df_join.filter(
    F.col("instituicao").isNull() &
    F.col("nome_entidade").isNotNull() &
    (F.trim(F.col("nome_entidade")) != "")
)

# Filtrar registros que estão presentes em ambos DataFrames
intersecao = df_join.filter(
    F.col("instituicao").isNotNull() &
    (F.trim(F.col("instituicao")) != "") &
    F.col("nome_entidade").isNotNull() &
    (F.trim(F.col("nome_entidade")) != "")
)
print(only_demonstracao_result.count())

print(only_info_cadastral.count())

print(intersecao.count())

print("total", df_join.count())


# testee = df_sem_acento.filter(F.col("nome_entidade") == "COOPERATIVA DE CRÉDITO RURAL COM INTERAÇÃO SOLIDÁRIA DE CASTRO - CRESOL CASTRO")

# table = tabulate(testee.collect(), headers=testee.columns, tablefmt='html')
# display(HTML(table))



24/07/18 17:09:42 WARN TaskSetManager: Stage 60 contains a task of very large size (12111 KiB). The maximum recommended task size is 1000 KiB.
24/07/18 17:09:44 WARN TaskSetManager: Stage 61 contains a task of very large size (11648 KiB). The maximum recommended task size is 1000 KiB.


0


24/07/18 17:09:48 WARN TaskSetManager: Stage 68 contains a task of very large size (12111 KiB). The maximum recommended task size is 1000 KiB.
24/07/18 17:09:49 WARN TaskSetManager: Stage 69 contains a task of very large size (11648 KiB). The maximum recommended task size is 1000 KiB.


2357


24/07/18 17:09:54 WARN TaskSetManager: Stage 77 contains a task of very large size (12111 KiB). The maximum recommended task size is 1000 KiB.
24/07/18 17:09:54 WARN TaskSetManager: Stage 78 contains a task of very large size (11648 KiB). The maximum recommended task size is 1000 KiB.


9698


24/07/18 17:10:00 WARN TaskSetManager: Stage 86 contains a task of very large size (12111 KiB). The maximum recommended task size is 1000 KiB.
24/07/18 17:10:00 WARN TaskSetManager: Stage 87 contains a task of very large size (11648 KiB). The maximum recommended task size is 1000 KiB.


total 12055


In [61]:
sample_only_demonstracao_result = only_demonstracao_result.sample(False, 0.1).limit(30)

tb_only_demonstracao_result = tabulate(only_demonstracao_result.collect(), headers=only_demonstracao_result.columns, tablefmt='html')

sample_only_info_cadastral = only_info_cadastral.sample(False, 0.1).limit(30)

tb_only_info_cadastral = tabulate(sample_only_info_cadastral.collect(), headers=sample_only_info_cadastral.columns, tablefmt='html')

sample_intersecao =intersecao.sample(False, 0.1).limit(30)

tb_intersecao = tabulate(sample_intersecao.collect(), headers=sample_intersecao.columns, tablefmt='html')

display(HTML(tb_only_demonstracao_result))
display(HTML(tb_only_info_cadastral))
display(HTML(tb_intersecao))



24/07/18 15:42:29 WARN TaskSetManager: Stage 371 contains a task of very large size (12111 KiB). The maximum recommended task size is 1000 KiB.
24/07/18 15:42:29 WARN TaskSetManager: Stage 372 contains a task of very large size (11648 KiB). The maximum recommended task size is 1000 KiB.
24/07/18 15:42:32 WARN TaskSetManager: Stage 377 contains a task of very large size (12111 KiB). The maximum recommended task size is 1000 KiB.
24/07/18 15:42:33 WARN TaskSetManager: Stage 378 contains a task of very large size (11648 KiB). The maximum recommended task size is 1000 KiB.
24/07/18 15:42:36 WARN TaskSetManager: Stage 383 contains a task of very large size (12111 KiB). The maximum recommended task size is 1000 KiB.
24/07/18 15:42:37 WARN TaskSetManager: Stage 384 contains a task of very large size (11648 KiB). The maximum recommended task size is 1000 KiB.


instituicao,tc,conglomerado,codigo,cnpj,data,cnpj_raiz,codigo_sisbacen,nome_tipo_entidade,descricao_situacao,nome_entidade,nome_fantasia
,,,,10664027000132,2024-05-23,10664027,,Sociedade Corretora de TVM,Cancelada/Encerrada,ABERTURA CORRETORA DE CAMBIO E VALORES MOBILIARIOS LTDA,
,,,,11547213000154,2024-05-23,11547213,,Administradora de Consórcio,Cancelada/Encerrada,ALMEIDA ADMINISTRACAO DE CONSORCIOS LTDA - IMPEDIDA CRIAR NOVOS GRUPOS,
,,,,33756370000102,2024-05-23,33756370,,Sociedade Corretora de TVM,Cancelada/Encerrada,"ARIJU S/A CORRETORA DE CAMBIO, TITULOS E VALORES MOBILIARIOS",
,,,,17338450000191,2024-05-23,17338450,,Sociedade Distribuidora de TVM,Regime Especial Encerrado,ASTRA DISTRIBUIDORA DE TITULOS E VALORES MOBILIARIOS LTDA,ASTRA DIST. TITLS. VALS. MOBILIARIOS LTD
,,,,38726063000100,2024-05-23,38726063,,Administradora de Consórcio,Cancelada/Encerrada,AUTO & MOVEL CONSORCIOS LTDA,
,,,,15208309000149,2024-05-23,15208309,,"Sociedade de Crédito, Financiamento e Investimento",Cancelada/Encerrada,"BAHIA LUCRO S/A - CREDITO, FINANCIAMENTO E INVESTIMENTOS",
,,,,61415428000190,2024-05-23,61415428,,Banco Múltiplo,Cancelada/Encerrada,BANCO DE TOKYO S.A.,
,,,,45666609000132,2024-05-23,45666609,,Banco Comercial Estrangeiro - Filial no país,Cancelada/Encerrada,BANCO HISPANO AMERICANO S.A.,
,,,,60700556000465,2024-05-23,60700556,,Banco Múltiplo,Cancelada/Encerrada,BANCO SANTANDER NOROESTE S.A.,
,,,,74419714000175,2024-05-23,74419714,,Banco de Investimento,Cancelada/Encerrada,BANCO SINTESE DE INVESTIMENTO S.A.,


instituicao,tc,conglomerado,codigo,cnpj,data,cnpj_raiz,codigo_sisbacen,nome_tipo_entidade,descricao_situacao,nome_entidade,nome_fantasia
QUANTO NETWORK INSTITUICAO DE PAGAMENTO LTDA.,2,QUANTO NETWORK IP - PRUDENCIAL,42064545,42064545000101,2024-05-23,42064545,00005,Instituição de Pagamento,Autorizada em Atividade,QUANTO NETWORK INSTITUICAO DE PAGAMENTO LTDA.,QUANTO
BLU INSTITUIÇÃO DE PAGAMENTO LTDA.,2,BLU IP - PRUDENCIAL,10506341,10506341000197,2024-05-23,10506341,00059,Instituição de Pagamento,Autorizada em Atividade,BLU INSTITUIÇÃO DE PAGAMENTO LTDA.,BLU IP
ZOOP TECNOLOGIA & INSTITUICAO DE PAGAMENTO S.A.,2,ZOOP TECNOLOGIA IP - PRUDENCIAL,19468242,19468242000213,2024-05-23,19468242,00137,Instituição de Pagamento,Autorizada em Atividade,ZOOP TECNOLOGIA & INSTITUICAO DE PAGAMENTO S.A.,ZOOP TECNOLOGIA & INSTITUIÇÃO DE PAGAMENTO S.A.
BANCO BANERJ S.A.,2,ITAU,33885724,33885724000119,2024-05-23,33885724,04029,Banco Múltiplo,Autorizada em Atividade,BANCO ITAÚ CONSIGNADO S.A.,BANCO ITAÚ CONSIGNADO S.A.
BANCO ITAÚ BMG CONSIGNADO S.A.,2,ITAU,33885724,33885724000119,2024-05-23,33885724,04029,Banco Múltiplo,Autorizada em Atividade,BANCO ITAÚ CONSIGNADO S.A.,BANCO ITAÚ CONSIGNADO S.A.
UBS PACTUAL CORRETORA DE TÍTULOS E VALORES MOBILIÁRIOS S.A.,3,UBS PACTUAL,43815158,43815158000122,2024-05-23,43815158,05211,Sociedade Corretora de TVM,Autorizada em Atividade,BTG PACTUAL CORRETORA DE TÍTULOS E VALORES MOBILIÁRIOS S.A.,BTG PACTUAL CTVM S.A.
BTG PACTUAL CORRETORA DE TÍTULOS E VALORES MOBILIÁRIOS S.A.,2,BTG PACTUAL,43815158,43815158000122,2024-05-23,43815158,05211,Sociedade Corretora de TVM,Autorizada em Atividade,BTG PACTUAL CORRETORA DE TÍTULOS E VALORES MOBILIÁRIOS S.A.,BTG PACTUAL CTVM S.A.
BANCO GE CAPITAL S.A.,2,BMG,62421979,62421979000129,2024-05-23,62421979,05233,Banco Múltiplo,Autorizada em Atividade,BANCO CIFRA S.A.,BANCO CIFRA
BANCO LETSBANK S.A.,2,BANCO VOITER - PRUDENCIAL,58497702,58497702000102,2024-05-23,58497702,05630,Banco Múltiplo,Autorizada em Atividade,BANCO LETSBANK S.A.,LETSBANK
BANCO ITAUBANK S.A.,2,ITAU - PRUDENCIAL,60394079,60394079000104,2024-05-23,60394079,06479,Banco Múltiplo,Autorizada em Atividade,BANCO ITAUBANK S.A.,ITAUBANK


In [13]:
df_join.printSchema()
df_join_sample = df_join.sample(False, 0.1).limit(30)

tb = tabulate(df_join_sample.collect(), headers=df_join_sample.columns, tablefmt='html')

display(HTML(tb))

root
 |-- instituicao: string (nullable = true)
 |-- tc: string (nullable = true)
 |-- conglomerado: string (nullable = true)
 |-- codigo: string (nullable = true)
 |-- cnpj: string (nullable = true)
 |-- data: string (nullable = true)
 |-- cnpj_raiz: string (nullable = true)
 |-- codigo_sisbacen: string (nullable = true)
 |-- nome_tipo_entidade: string (nullable = true)
 |-- descricao_situacao: string (nullable = true)
 |-- nome_entidade: string (nullable = true)
 |-- nome_fantasia: string (nullable = true)



24/07/18 17:15:04 WARN TaskSetManager: Stage 110 contains a task of very large size (12111 KiB). The maximum recommended task size is 1000 KiB.
24/07/18 17:15:04 WARN TaskSetManager: Stage 111 contains a task of very large size (11648 KiB). The maximum recommended task size is 1000 KiB.


instituicao,tc,conglomerado,codigo,cnpj,data,cnpj_raiz,codigo_sisbacen,nome_tipo_entidade,descricao_situacao,nome_entidade,nome_fantasia
CATERPILLAR FINANCIAL S.A. ARRENDAMENTO MERCANTIL,3,CATERPILLAR,02658435,02658435000153,2024-05-23,02658435,53154,Banco Múltiplo,Autorizada em Atividade,BANCO CATERPILLAR S.A.,BANCO CATERPILLAR S.A.
BANCO PRIME DE INVESTIMENTO S.A.,2,PRIME,45246410,45246410000155,2024-05-23,45246410,31710,Banco Múltiplo,Autorizada em Atividade,BANCO GENIAL S.A.,PLURAL BANCO
CM CAPITAL MARKETS CORRETORA DE CAMBIO LTDA,3,CM CAPITAL MARKETS,02671743,02671743000119,2024-06-12,02671743,57538,Sociedade Distribuidora de TVM,Autorizada em Atividade,BANVOX DISTRIBUIDORA DE TÍTULOS E VALORES MOBILIÁRIOS LTDA,BANVOX DTVM
BANCO FINAXIS S.A.,2,FINAXIS - PRUDENCIAL,11758741,11758741000152,2024-05-23,11758741,27731,Banco Comercial,Autorizada em Atividade,BANCO FINAXIS S.A.,BANCO FINAXIS S.A.
BANCO INDUSVAL S.A.,2,INDUSVAL,61024352,61024352000171,2024-05-23,61024352,05653,Banco Múltiplo,Autorizada em Atividade,BANCO VOITER S.A.,VOITER
BANKPAR BANCO MÚLTIPLO S.A.,2,BRADESCO,59438325,59438325000101,2024-05-23,59438325,07204,Banco Múltiplo,Cancelada/Encerrada,BANCO BRADESCO CARTÕES S.A.,BRADESCO CARTÕES
STONE SOCIEDADE DE CRÉDITO DIRETO S.A.,2,STONE PAGAMENTOS - PRUDENCIAL,34590184,34590184000109,2024-05-23,34590184,42504,Sociedade de Crédito Direto,Autorizada em Atividade,STONE SOCIEDADE DE CRÉDITO DIRETO S.A.,STONE SCD
SAFRA DISTRIBUIDORA DE TITULOS E VALORES MOBILIARIOS LTDA,2,SAFRA,62180047,62180047000131,2024-05-23,62180047,37383,Sociedade Distribuidora de TVM,Cancelada/Encerrada,SAFRA ASSET MANAGEMENT LTDA,SAFRA ASSET MANAGEMENT
M18 INSTITUIÇÃO DE PAGAMENTO S.A.,2,M18 IP - PRUDENCIAL,22575466,22575466000158,2024-05-23,22575466,41921,Instituição de Pagamento,Autorizada em Atividade,SRM BANK INSTITUIÇÃO DE PAGAMENTO S/A,SRM BANK
SANTANDER SECURITIES (BRASIL)CORRETORA DE VALORES MOBILIÁRIOS S.A.,3,SANTANDER,61733515,61733515000195,2024-05-23,61733515,57054,Sociedade Corretora de TVM,Cancelada/Encerrada,SANTANDER SECURITIES (BRASIL)CORRETORA DE VALORES MOBILIÁRIOS S.A.,SANTANDER SECURITIES CVM


['codigo_cadastro_bacen', 'codigo_sisbacen', 'codigo_pais_sede', 'nome_pais_sede', 'nome_uf_sede', 'codigo_municipio_sede', 'nome_municipio_sede', 'nome_entidade', 'nome_entidade_nao_formatado', 'cnpj', 'cnpj_raiz', 'codigo_situacao', 'descricao_situacao', 'codigo_tipo_entidade_segmento', 'nome_tipo_entidade', 'codigo_natureza_juridica', 'descricao_natureza_juridica', 'codigo_esfera_publica', 'nome_reduzido', 'sigla_entidade', 'nome_fantasia', 'empresa_publica']
<class 'list'>


In [58]:
data = [("1",), ("123",), ("12345",), ("12345678",)]
columns = ["codigo"]
df = spark.createDataFrame(data, columns)

# Adicionar zeros à esquerda para que todos os valores na coluna "codigo" tenham exatamente 8 dígitos
df_formatado = df.withColumn("codigo", F.lpad(df["codigo"], 8, '0'))

# Exibir o DataFrame formatado
df_formatado.show(truncate=False)

+--------+
|codigo  |
+--------+
|00000001|
|00000123|
|00012345|
|12345678|
+--------+

